# Tokenización de texto para calcular costos de uso de OpenAI

Fuente: [How_to_count_tokens_with_tiktoken - OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [3]:
import tiktoken
from tiktoken import Encoding

## Selección y carga de encoding 

`cl100k_base` para modelos `gpt-4, gpt-3.5-turbo, text-embedding-ada-002`.

Consulta el [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb) para otro tipo de modelos o cambios en este encoding.

In [4]:
encoding: Encoding = tiktoken.get_encoding("o200k_base")

In [5]:
encoding_model: Encoding = tiktoken.encoding_for_model("gpt-4o-mini")

## Tokenización de string

In [7]:
encoding_model.encode("El perrito come solito.")

[4422, 777, 33139, 3063, 1887, 2955, 13]

In [8]:
len(encoding_model.encode("El perrito come solito."))

7

## Tokenizar una serie de mensajes

In [9]:
def num_tokens_from_messages(messages, model="gpt-4o-mini-2024-07-18"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding: Encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using o200k_base encoding.")
        encoding = tiktoken.get_encoding("o200k_base")
    if model in {
        "gpt-3.5-turbo-0125",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        "gpt-4o-mini-2024-07-18",
        "gpt-4o-2024-08-06"
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0125.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0125")
    elif "gpt-4o-mini" in model:
        print("Warning: gpt-4o-mini may update over time. Returning num tokens assuming gpt-4o-mini-2024-07-18.")
        return num_tokens_from_messages(messages, model="gpt-4o-mini-2024-07-18")
    elif "gpt-4o" in model:
        print("Warning: gpt-4o and gpt-4o-mini may update over time. Returning num tokens assuming gpt-4o-2024-08-06.")
        return num_tokens_from_messages(messages, model="gpt-4o-2024-08-06")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [11]:
example_messages: list[dict[str, str]] = [
    {
      "role": "system",
      "content": "Eres un asistente de atención a clientes y estudiantes de la plataforma de educación online en tecnología, inglés y liderazgo llamada Platzi"
    },
    {
      "role": "user",
      "content": "¿Cómo puedo mejorar mis habilidades de creación de contenido con inteligencia artificial?"
    },
    {
      "role": "assistant",
      "content": "Para mejorar tus habilidades de creación de contenido con inteligencia artificial te sugiero que tomes el Taller de Creación de Contenido con Inteligencia Artificial en https://platzi.com/cursos/contenido-ia/. En este curso aprenderás a utilizar herramientas para generar contenido de manera automatizada y eficiente. ¡No te lo pierdas!"
    }
]

In [12]:
num_tokens_from_messages(example_messages, model="gpt-3.5-turbo-0125")

139